In [0]:
from pyspark.sql import functions as F

# Load Bronze ratings
df_ratings = spark.table("bronze_ratings_csv")

# Cast rating to float and timestamp if needed
df_ratings_silver = df_ratings.withColumn("rating", F.col("rating").cast("float"))

# Optional: parse timestamp if there is a timestamp column
df_ratings_silver = df_ratings_silver.withColumn(
    "timestamp_ts",
    F.from_unixtime(F.col("timestamp").cast("long")).cast("timestamp")
)

# Remove any duplicates
df_ratings_silver = df_ratings_silver.dropDuplicates(["userId", "movieId"])

# Quick check
df_ratings_silver.show(5)
df_ratings_silver.printSchema()


+------+-------+------+---------+-------------------+
|userId|movieId|rating|timestamp|       timestamp_ts|
+------+-------+------+---------+-------------------+
|     1|      1|   4.0|964982703|2000-07-30 18:45:03|
|     1|      3|   4.0|964981247|2000-07-30 18:20:47|
|     1|      6|   4.0|964982224|2000-07-30 18:37:04|
|     1|     47|   5.0|964983815|2000-07-30 19:03:35|
|     1|     50|   5.0|964982931|2000-07-30 18:48:51|
+------+-------+------+---------+-------------------+
only showing top 5 rows
root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- timestamp_ts: timestamp (nullable = true)



In [0]:
df_movies = spark.table("bronze_movies_csv")

# Convert genres string to array
df_movies_silver = df_movies.withColumn("genres_array", F.split(F.col("genres"), "\\|"))

# Remove duplicates by movieId just in case
df_movies_silver = df_movies_silver.dropDuplicates(["movieId"])

df_movies_silver.show(5, truncate=False)
df_movies_silver.printSchema()


+-------+----------------------------------+-------------------------------------------+-------------------------------------------------+
|movieId|title                             |genres                                     |genres_array                                     |
+-------+----------------------------------+-------------------------------------------+-------------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|[Adventure, Animation, Children, Comedy, Fantasy]|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |[Adventure, Children, Fantasy]                   |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |[Comedy, Romance]                                |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |[Comedy, Drama, Romance]                         |
|5      |Father of the Brid

In [0]:
df_links = spark.table("bronze_links_csv")

df_movies_silver = df_movies_silver.join(df_links, "movieId", "left")

df_movies_silver.show(5, truncate=False)


+-------+-----------------------------------+--------------------+------------------------+-------+------+
|movieId|title                              |genres              |genres_array            |imdbId |tmdbId|
+-------+-----------------------------------+--------------------+------------------------+-------+------+
|73     |Misérables, Les (1995)             |Drama|War           |[Drama, War]            |0113828|48750 |
|110    |Braveheart (1995)                  |Action|Drama|War    |[Action, Drama, War]    |0112573|197   |
|146    |Amazing Panda Adventure, The (1995)|Adventure|Children  |[Adventure, Children]   |0112342|30765 |
|149    |Amateur (1994)                     |Crime|Drama|Thriller|[Crime, Drama, Thriller]|0109093|30157 |
|155    |Beyond Rangoon (1995)              |Adventure|Drama|War |[Adventure, Drama, War] |0112495|1873  |
+-------+-----------------------------------+--------------------+------------------------+-------+------+
only showing top 5 rows


In [0]:
# Load movies and links
df_movies = spark.table("bronze_movies_csv")
df_links = spark.table("bronze_links_csv")

from pyspark.sql import functions as F

df_movies_silver = (
    df_movies
    .withColumn("genres_array", F.split(F.col("genres"), "\\|"))
    .dropDuplicates(["movieId"])
    .join(df_links, "movieId", "left")  # keep imdbId if needed later
)

# Optional: save as permanent Delta table
df_movies_silver.write.format("delta").mode("overwrite").saveAsTable("silver_dim_movies_enriched")


In [0]:
df_ratings_silver = spark.table("bronze_ratings_csv") \
    .withColumn("rating", F.col("rating").cast("float")) \
    .withColumn("timestamp_ts", F.from_unixtime(F.col("timestamp").cast("long")).cast("timestamp")) \
    .dropDuplicates(["userId", "movieId"])

df_ratings_silver.write.format("delta").mode("overwrite").saveAsTable("silver_fact_ratings")


In [0]:
df_dim_users = df_ratings_silver.groupBy("userId").agg(
    F.count("movieId").alias("num_ratings"),
    F.avg("rating").alias("avg_rating")
)

df_dim_users.write.format("delta").mode("overwrite").saveAsTable("silver_dim_users")


In [0]:
tables = spark.catalog.listTables()
for t in tables:
    if t.name.startswith("silver_"):
        print(f"{t.name}  |  {t.tableType}  |  {t.isTemporary}")


silver_dim_movies_enriched  |  MANAGED  |  False
silver_dim_users  |  MANAGED  |  False
silver_fact_ratings  |  MANAGED  |  False


In [0]:
# List all managed tables in the current database
all_tables = spark.catalog.listTables()

# Filter for tables starting with 'bronze' or 'silver'
bronze_silver_tables = [
    t.name for t in all_tables
    if t.tableType == "MANAGED" and not t.isTemporary and (t.name.startswith("bronze") or t.name.startswith("silver"))
]

print("Tables starting with 'bronze' or 'silver':")
for t in bronze_silver_tables:
    print(t)


Tables starting with 'bronze' or 'silver':
bronze_links_csv
bronze_movies_csv
bronze_ratings_csv
silver_dim_movies_enriched
silver_dim_users
silver_fact_ratings
